In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import tlc

In [68]:
dataset_name = "cats-and-dogs"
project_name = "image-classification-table"
table_name = "initial"

table = tlc.Table.from_names(    
    table_name="initial",
    dataset_name=dataset_name,
    project_name=project_name,
)

In [69]:
from tools.split import split_table

splits = split_table(
    table=table,
    splits={"train": 0.7, "val": 0.3},
    split_strategy="stratified",
    stratify_column=1,
)

train_table = splits["train"]
val_table = splits["val"]

3lc: Creating transaction
3lc: Committing transaction
3lc: Creating transaction
3lc: Committing transaction


In [70]:
print(len(train_table))
print(len(val_table))

7
3


In [71]:
for row in train_table:
    print(row)

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=350x280 at 0x321D93470>, 0)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x480 at 0x321D93080>, 0)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=387x500 at 0x321D93470>, 0)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=407x282 at 0x321D93080>, 1)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x321D93470>, 1)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=420x500 at 0x321D93080>, 1)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=395x360 at 0x321D93470>, 1)


In [72]:
for row in val_table:
    print(row)

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=491x500 at 0x321D93B60>, 0)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x321D93470>, 0)
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=320x240 at 0x321D93290>, 1)
